# Tutorial: Play Bird Audio on Map with Folium
Really enjoying playing with audio data for the first time, thought I'd share a cool way I found to play the sounds of the birds based on their locations directly on a map!

This tutorial uses [Folium](https://python-visualization.github.io/folium/), a Python Library for plotting geospatial data. It's Python wrapper for Leaflet.js (a leading open-source JavaScript library for plotting interactive maps). It has the power of Leaflet.js but you get to use Python, so it's perfect for this use case!

# Setup

In [ ]:
#imports
import pandas as pd
import numpy as np
import random
import time

import folium

In [ ]:
#get data
path = '/kaggle/input/birdclef-2022/'
train_metadata = pd.read_csv(path + 'train_metadata.csv')

In [ ]:
#because we have some duplicate latitutde/longitutde points, let's fuzzy them up a bit to avoid map overlap
def fuzzy_lat_long(num):
    """Add random 4 digit float to latitude and longitude betwen 0-0.00009. Assumes random is imported.  
    
    Args:    num (float): the latitude or longitude value to 4 decimal places
    
    Returns:    new_num (float): updated latitude or longitude value to 8 decimal places"""

    random_value = round(random.uniform(0, 0.00009),8)
    new_num = num + random_value
    
    return new_num

In [ ]:
#fuzzy it up
train_metadata['newlat'] = train_metadata.apply(lambda x: fuzzy_lat_long(x['latitude']), axis=1)
train_metadata['newlong'] = train_metadata.apply(lambda x: fuzzy_lat_long(x['longitude']), axis=1)

In [ ]:
#Let's also visualize only the scored_birds in the map to see what it looks like
scored_birds = ['akiapo', 'aniani', 'apapan', 'barpet', 'crehon', 'elepai', 'ercfra', 'hawama', 'hawcre', 'hawgoo', 'hawhaw', 'hawpet1', 'houfin', 'iiwi', 'jabwar', 'maupar', 'omao', 'puaioh', 'skylar', 'warwhe1', 'yefcan']
train = train_metadata[train_metadata['primary_label'].isin(scored_birds)]

In [ ]:
train.head()

In [ ]:
train.shape

## Fun with Folium
* In just a few lines of code, create a useful/exciting tool to listen to bird audio by location!

In [ ]:
#construct folium birdmap
birdmap = folium.Map(
    location = [19.629425,-155.361479], #this is just a reference point to start the map --> for reference it's in Hawaii.
    zoom_start = 8 #this is how zoomed in you are on that start point. 
    #zoom_start = 3 #change the zoom_start to 3 to see how it changes the reference frame of the map below!
) 

* We now have the map started up. Next is to populate it.
* We populate the map by going row by row through out train dataframe.
* The folium popup is html so you can do anything you can think of to do with html code. 
 * I embeded the sounds from the urls provided, but you can do whatever you want in there!
 * Also because we're going in a loop I just pulled in some values like common_name, location (latitude, longitude), and primary_label.

In [ ]:
for row in train.iterrows():
    #base values
    row_values = row[1] #this gets the 
    soundsrc = str(row_values['url']) + '/embed?simple=1' #this is the string part of the embed code...
    location = [row_values['newlat'], row_values['newlong']] 
    
    #html popup
    popup = '</iframe>' \
            f'<strong>  {row_values["common_name"]}  </strong> \
            <br><br> \
            <strong> Location: </strong> {location}  \
            <br><br> \
            <strong> Rating: </strong> {row_values["rating"]} \
            <br><br> \
            This is a: <strong> {row_values["primary_label"]} </strong>... Listen to it below. \
            <br><br> \
            <iframe src={soundsrc} scrolling="no" frameborder="0" width="340" height="115"></iframe>' \
            '</iframe>'
    marker = folium.Marker(location=location, popup = popup) #defines the marker
    marker.add_to(birdmap) #adds the marker to the map

## Finished Product: Birdmap !

### 1. Zoom in and Out
### 2. Click on Blue Icon to find out more about the recording and play the audio of that recording!

In [ ]:
#click on a blue marker to see the common name, location of sound recording, primary_label, rating, and play audio!
display(birdmap)

### Hope you enjoyed this tutorial!

### For another visualization technique, check out: [Tutorial: BirdCLEF Visualizations with Plotly](https://www.kaggle.com/alexteboul/tutorial-birdclef-visualizations-with-plotly)

